# Yolov8 Model Training

This notebook helps easily train a yolo model. Model evaluation will happen in another notebook.

## 1. Imports and variable setup

In [1]:
# Import all libraries
import os
from roboflow import Roboflow
from IPython import display
import ultralytics
from ultralytics import YOLO
from IPython.display import display, Image
from pathlib import Path
from dotenv import find_dotenv, load_dotenv
import sys
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch

# Setup HOME environment variable
HOME = os.getcwd()
print(HOME)

if torch.cuda.is_available():
    Device = torch.device("cuda")
elif torch.backends.mps.is_available():
    Device = torch.device("mps")
else:
    Device = torch.device("cpu")
print(Device)

/home/mreag/repos/DBD-Killer-AI/notebooks
cuda


In [3]:
# Check image displays are good
# display.clear_output()
sys.path.append(str(Path.cwd().parent))
# Check ultralytics library is good
ultralytics.checks()

Ultralytics YOLOv8.0.238 🚀 Python-3.10.13 torch-2.4.1 CPU (Apple M1 Pro)
Setup complete ✅ (8 CPUs, 16.0 GB RAM, 306.4/460.4 GB disk)


In [4]:
# Environment variables
PROJECT_DIR = Path.cwd().parent
DATASET_VERSION = 8

## 2. Import Model

Pre-trained is the ideal model

In [5]:
model = YOLO(model=str(PROJECT_DIR) + "/models/yolov8n.pt")

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


## 3. Train Model

In [6]:
str(PROJECT_DIR) + "/data/external"

'/Users/mreagles524/Documents/gitrepos/projects/DBD-Killer-AI/data/external'

In [8]:
from dbdkillerai.data.make_dataset import roboflow_connect, roboflow_download
from dbdkillerai.models.train_model import train_yolo, validate_yolo

# Establish Roboflow connection and acquire dataset location. DONT download.
rf_conn, rf_project = roboflow_connect()
data_location = roboflow_download(rf_project=rf_project,
                                  rf_data_version=DATASET_VERSION,
                                  data_format="yolov8",
                                  project_dir=str(PROJECT_DIR) + "/data/external",
                                  overwrite=True)

# Get location of yml
yml_location = data_location.location + "/data.yaml"
model, results_train = train_yolo(yolo_model=model,
                                    data_yml=yml_location,
                                    epochs=150,
                                    imgsz=800,
                                    plots=True,
                                    workers=0,
                                    device="mps")



loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.238, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to /Users/mreagles524/Documents/gitrepos/projects/DBD-Killer-AI/data/external/deadbydaylightkillerai/killer_ai_object_detection/8 in yolov8:: 100%|██████████| 5702/5702 [00:00<00:00, 9339.14it/s]


New https://pypi.org/project/ultralytics/8.3.36 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.238 🚀 Python-3.10.13 torch-2.4.1 MPS (Apple M1 Pro)
engine/trainer: task=detect, mode=train, model=/Users/mreagles524/Documents/gitrepos/projects/DBD-Killer-AI/models/yolov8n.pt, data=/Users/mreagles524/Documents/gitrepos/projects/DBD-Killer-AI/data/external/deadbydaylightkillerai/killer_ai_object_detection/8/data.yaml, epochs=10, time=None, patience=50, batch=16, imgsz=800, save=True, save_period=-1, cache=False, device=mps, workers=0, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, so

train: Scanning /Users/mreagles524/Documents/gitrepos/projects/DBD-Killer-AI/data/external/deadbydaylightkillerai/killer_ai_object_detection/8/train/labels.cache... 2488 images, 593 backgrounds, 0 corrupt: 100%|██████████| 2488/2488 [00:00<?, ?it/s]
val: Scanning /Users/mreagles524/Documents/gitrepos/projects/DBD-Killer-AI/data/external/deadbydaylightkillerai/killer_ai_object_detection/8/valid/labels.cache... 238 images, 58 backgrounds, 0 corrupt: 100%|██████████| 238/238 [00:00<?, ?it/s]


Plotting labels to runs/detect/train5/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10         0G       1.57       5.04      1.374         11        800: 100%|██████████| 156/156 [03:50<00:00,  1.48s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/8 [00:08<?, ?it/s]


RuntimeError: Couldn't load custom C++ ops. This can happen if your PyTorch and torchvision versions are incompatible, or if you had errors while compiling torchvision from source. For further information on the compatible versions, check https://github.com/pytorch/vision#installation for the compatibility matrix. Please check your PyTorch version with torch.__version__ and your torchvision version with torchvision.__version__ and verify if they are compatible, and if not please reinstall torchvision so that it matches your PyTorch install.

## Validate Custom Model

In [ ]:
best_weights_location = str(results_train.save_dir) + "/weights/best.pt"
best_model = YOLO(best_weights_location)
results_val = validate_yolo(yolo_model=best_model,
                            data_yml=yml_location)


Ultralytics YOLOv8.0.227 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
Model summary (fused): 168 layers, 3006428 parameters, 0 gradients


val: Scanning /home/mreag/repos/DBD-Killer-AI/data/external/deadbydaylightkillerai/killer_ai_object_detection/7/valid/labels.cache... 238 images, 80 backgrounds, 0 corrupt: 100%|██████████| 238/238 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:02<00:00,  6.26it/s]


                   all        238        343      0.806      0.713      0.765       0.52
              activity        238         12      0.815      0.733      0.777      0.501
             generator        238        189      0.862      0.884      0.904      0.656
                  hook        238        120      0.855      0.734      0.791      0.526
              survivor        238         22      0.694        0.5      0.588      0.396
Speed: 0.4ms preprocess, 2.8ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/val5
